In [2]:
import pandas as pd
import numpy as np

In [36]:
class BELT_Analyzer2:
    def __init__(self, main_csv_path, log_path):
        self.main_csv_path = main_csv_path
        self.log_path      = log_path
        
        self.data_main     = pd.read_csv(self.main_csv_path)
        self.data_log      = self.readLog(self.log_path)
        self.data_main_ext = self.setResponseTimeOnMainFromLog(self.data_log)
    
    '''
    computeResTime:
        Get an array containing a sequence of timestamp.
        Return an array containing the time differences.
        e.g.) input <- [1,4,12,13]
              output <- [3,8,1]
    '''
    def computeResTime(self, timelist):
        diff = timelist[1:] - timelist[:-1]
        return diff
    
    
    '''
    getBalloonPointsAndPops:

    '''
    def getBalloonPointsAndPops(self, dataframe):
        # blueballoon
        blueballoon_points = np.sum(dataframe[dataframe['imgroot']=='blueballoon']['balloonscore'].values)
        blueballoon_pops   = len(np.argwhere(dataframe[dataframe['imgroot']=='blueballoon']['balloonscore'].values==0))
        # pinkballoon
        pinkballoon_points = np.sum(dataframe[dataframe['imgroot']=='pinkballoon']['balloonscore'].values)
        pinkballoon_pops   = len(np.argwhere(dataframe[dataframe['imgroot']=='pinkballoon']['balloonscore'].values==0))
        # orangeballoon
        orangeballoon_points = np.sum(dataframe[dataframe['imgroot']=='orangeballoon']['balloonscore'].values)
        orangeballoon_pops   = len(np.argwhere(dataframe[dataframe['imgroot']=='orangeballoon']['balloonscore'].values==0))
        result = {"blue_score"  : [blueballoon_points],
                  "blue_pops"   : [blueballoon_pops],
                  "pink_score"  : [pinkballoon_points],
                  "pink_pops"   : [pinkballoon_pops],
                  "orange_score": [orangeballoon_points],
                  "orange_pops" : [orangeballoon_pops]}
        return result

    
    '''
    readLog:
    '''
    def readLog(self,log_path):
        # Read .log file and store data into dataframe
        with open(log_path, mode='r', encoding='UTF-8') as f:
            time_list = []
            type_list = []
            mesg_list = []
            for line in f:
                columns = line.split("\t")
                time_list.append(float(columns[0].rstrip()))
                type_list.append(columns[1].rstrip())
                mesg_list.append(columns[2].rstrip())
        data_json = {'timestamp':time_list, 'datatype':type_list, 'msg':mesg_list}
        data_log = pd.DataFrame(data_json)
        return data_log
    
    '''
    setResponseTimeFromLog:

    '''
    def setResponseTimeOnMainFromLog(self, data_log):
        # This list will contain the average reaction(response) time of each trial (balloon)
        out_avg_rxn_time = []
        
        _timestamp = []
        # Iterate log data, for each new trial, collect timestamp when a participant hit a button.
        for index, row in data_log.iterrows():
            # New balloon is shown. Empty timestamp buffer.
            if row['msg'].startswith('New trial'):
                # Corner case: Very first trial of experiment.
                if(len(_timestamp)==0):
                    pass
                else:
                    _restime_arr = self.computeResTime(np.array(_timestamp))
                    #print("Score: {} Avg_rxn_time: {}".format(len(_restime_arr), np.mean(_restime_arr)))
                    out_avg_rxn_time.append(np.mean(_restime_arr))
                    _timestamp = []
                _timestamp.append(row['timestamp'])
            if row['msg'].startswith('Keypress: space'):
                _timestamp.append(row['timestamp'])
        _restime_arr = self.computeResTime(np.array(_timestamp[:-1]))
        #print("Score: {} Avg_rxn_time: {}".format(len(_restime_arr), np.mean(_restime_arr)))
        out_avg_rxn_time.append(np.mean(_restime_arr))

        self.data_main["avgRxnTime"] = out_avg_rxn_time

  


In [37]:
# Read main.csv
csv_path = './data/AA06LC00_BELT_TEST_2021_Jun_09_1320.csv'
# Set .log path
log_path = './data/AA06LC00_BELT_TEST_2021_Jun_09_1320.log'


my_BELT = BELT_Analyzer2(csv_path, log_path)



In [40]:
# average reaction time after popped balloons*
popped_balloons_trial_idx = np.argwhere(my_BELT.data_main['balloonscore'].values==0).squeeze()
if(len(popped_balloons_trial_idx)<1):
    print("[INFO] No popped balloons")
else:
    popped_balloons_trial_idx += 1 # increase index to target the AFTER popped
    # The last balloon is popped. In this case, there is no more following trial to compute, thus we ignore the last pop.
    if(popped_balloons_trial_idx[-1]==len(my_BELT.data_main)-1):
        print("[INFO] A balloon is poppped at the last trial. Thus, unable to compute avg reaction time AFTER popped balloon.")
        popped_balloons_trial_idx = popped_balloons_trial_idx[:-1]
    avg_rxntime_after_popped = np.mean(my_BELT.data_main.iloc[np.argwhere(my_BELT.data_main['balloonscore'].values==0).squeeze() + 1].avgRxnTime)


In [6]:

# points on balloons by color (condition)
points_on_balloons_per_color = my_BELT.data_main.groupby('imgroot').mean().balloonscore

# number of points for each participant
tot_balloonscore = np.sum(my_BELT.data_main['balloonscore'].values)

# number of pops per participant*
tot_pops = len(np.argwhere(my_BELT.data_main['balloonscore'].values==0))

# number of pops and number of points per color condition overall*
## first half
data_main_first_half = my_BELT.data_main[my_BELT.data_main['trials.thisRepN']==0]
points_pops_per_color_first_half = my_BELT.getBalloonPointsAndPops(data_main_first_half)
## second half
data_main_second_half = my_BELT.data_main[my_BELT.data_main['trials.thisRepN']==1]
points_pops_per_color_second_half = my_BELT.getBalloonPointsAndPops(data_main_second_half)

# average reaction time after popped balloons*
popped_balloons_trial_idx = np.argwhere(my_BELT.data_main['balloonscore'].values==0).squeeze()
if(len(popped_balloons_trial_idx)<1):
    print("[INFO] No popped balloons")
else:
    popped_balloons_trial_idx += 1 # increase index to target the AFTER popped
    # The last balloon is popped. In this case, there is no more following trial to compute, thus we ignore the last pop.
    if(popped_balloons_trial_idx[-1]==len(my_BELT.data_main)-1):
        print("[INFO] A balloon is poppped at the last trial. Thus, unable to compute avg reaction time AFTER popped balloon.")
        popped_balloons_trial_idx = popped_balloons_trial_idx[:-1]
    avg_rxntime_after_popped = np.mean(my_BELT.data_main.iloc[np.argwhere(my_BELT.data_main['balloonscore'].values==0).squeeze() + 1].avgRxnTime)

# average reaction time by color
blue, orange, pink = my_BELT.data_main.groupby('imgroot').mean().avgRxnTime
# split blue balloons into load sizes (there are three) with average reaction time for each
blue_7, blue_13, blue_19 = my_BELT.data_main[my_BELT.data_main['imgroot']=='blueballoon'].groupby('maxpumps').mean().avgRxnTime

In [15]:
import os
subject_id = os.path.basename(csv_path).split('_')[0]

In [16]:
subject_id

'AA06LC00'

In [27]:
pd.DataFrame({"subjectId": [subject_id], "total_balloonscore":my_BELT.data_main.sum().balloonscore})

,subjectId,total_balloonscore
0,AA06LC00,440


In [46]:
my_BELT.data_main

,maxpumps,imgroot,trials.thisRepN,trials.thisTrialN,trials.thisN,trials.thisIndex,nPumps,balloonscore,date,frameRate,expName,participant,Unnamed: 12,avgRxnTime
0,19,pinkballoon,0,0,0,0,11,11,2021_Jun_09_1320,60.356842,BELT_TEST,AA06LC00,NaN,1.122036
1,19,pinkballoon,0,1,1,1,13,13,2021_Jun_09_1320,60.356842,BELT_TEST,AA06LC00,NaN,1.314485
2,19,blueballoon,0,2,2,2,12,12,2021_Jun_09_1320,60.356842,BELT_TEST,AA06LC00,NaN,1.218917
3,7,blueballoon,0,3,3,3,8,0,2021_Jun_09_1320,60.356842,BELT_TEST,AA06LC00,NaN,1.052688
4,19,pinkballoon,0,4,4,4,10,10,2021_Jun_09_1320,60.356842,BELT_TEST,AA06LC00,NaN,1.058160
5,13,blueballoon,0,5,5,5,8,8,2021_Jun_09_1320,60.356842,BELT_TEST,AA06LC00,NaN,0.898163
6,7,orangeballoon,0,6,6,6,8,0,2021_Jun_09_1320,60.356842,BELT_TEST,AA06LC00,NaN,0.861387
7,7,orangeballoon,0,7,7,7,8,0,2021_Jun_09_1320,60.356842,BELT_TEST,AA06LC00,NaN,0.802675
8,7,orangeballoon,0,8,8,8,5,5,2021_Jun_09_1320,60.356842,BELT_TEST,AA06LC00,NaN,0.666560
9,19,pinkballoon,0,9,9,9,13,13,2021_Jun_09_1320,60.356842,BELT_TEST,AA06LC00,NaN,1.182685


In [ ]:
+--*---*--*-----*--*---+----*--*--*-*---+

# response time per presentation from onset stimulus
# response time from previous stimulus
# [Done] points on balloons by color (condition)
# [Done] reaction time on average after popped balloons*
# [Done] number of points for each participant
# [Done] number of pops per participant*
# [Done] number of pops and number of points per color condition overall*
## [Done] number of pops and points per color condition on 1st half (trials 1-27)*
## [Done] number of pops and points per color condition on 2nd half (trials 28-56)*
# [Done] average reaction time after popped balloons*
# [Done] average reaction time by color
# [Done] split blue balloons into load sizes (there are three) with average reaction time for each
*note on every calculation that involves pops: Pops with number of pumps that do not exceed 7 are not actually pumps, but actually a skipped trial due to hitting enter too fast. Please do not count these as pops.
 